<a href="https://colab.research.google.com/github/kexinz8/fashionMNIST/blob/main/stat542_proj_no_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

from sklearn import metrics

# Import data

In [ ]:
# import data
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

In [6]:
train_x = train_data.reshape(train_data.shape[0], -1).astype('float32')
test_x = test_data.reshape(test_data.shape[0], -1).astype('float32')

# Multiclassification
## LightGBM

In [8]:
from lightgbm import LGBMClassifier

In [10]:
# dont run this part, i'm still tuning
err_list = []
for i in np.arange(0.0, 1.0, 0.1):
    model = LGBMClassifier(objective='multiclass',path_smooth = i)
    model.fit(train_x, train_labels, categorical_feature=[0,3])
    predictions = model.predict(test_x)
    error = sum(predictions!=test_x)/len(test_labels)
    err_list.append(error)
err_list

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 3]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


KeyboardInterrupt: ignored

In [ ]:
lgb_model = LGBMClassifier(objective='multiclass',path_smooth = 0.5)
lgb_model.fit(train_x,train_labels,categorical_feature=[0,3])

In [ ]:
expected_y  = test_labels
predicted_y = lgb_model.predict(test_x)
lightGBM_pred_y = predicted_y
print(metrics.classification_report(expected_y, predicted_y))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
tune = np.arange(0.0, 1.0, 0.1)
error = np.array(err_list)

plt.style.use("fivethirtyeight")
plt.plot(tune, error)

plt.xlabel("Misclassification Error Rate")
plt.ylabel("Path Smooth")

plt.show()

## LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
lda = LDA(n_components=9)

In [ ]:
expected_y  = test_labels
lda_model = lda.fit(train_x, train_labels)
predicted_y = lda_model.predict(test_x)
lda_pred_y = predicted_y
print(metrics.classification_report(expected_y, predicted_y))

In [ ]:
report = metrics.classification_report(expected_y, predicted_y, output_dict=True)
pd.DataFrame(report).transpose()

## SVM

In [ ]:
from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
# Use Ploynomial kernel 
svmpoly = svm.SVC(kernel='poly', degree=3, C=1).fit(train_x, train_labels)

In [ ]:
expected_y  = test_labels
predicted_y = svmpoly.predict(test_x)
SVM_pred_y = predicted_y
print(metrics.classification_report(expected_y, predicted_y))

In [ ]:
report = metrics.classification_report(expected_y, predicted_y, output_dict=True)
pd.DataFrame(report).transpose()

Combine the three models results

In [ ]:
# to get the precision dictionary
out = {}
for i in range(0,10):
    out.update({i:round(report[str(i)]['precision'],2)})

In [ ]:
# create df of y_pred of test set for each model
y_pred_test = pd.concat([pd.DataFrame(lightGBM_pred_y, columns = ['LightGBM']),
           pd.DataFrame(lda_pred_y, columns = ['LDA']), 
           pd.DataFrame(SVM_pred_y, columns = ['SVM'])],axis=1)

In [ ]:
y_pred_test.to_csv('y_pred_test.csv')

# Ensemble


In [2]:
import lightgbm as ltg
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier

In [3]:
estimator = []
estimator.append(('LDA', LDA(n_components=9)))
estimator.append(('SVC', svm.SVC(kernel='poly', degree=3, C=1)))
estimator.append(('LGBM', ltg.LGBMClassifier(objective='multiclass',path_smooth = 0.2)))

## Voting

In [7]:
vot_hard=VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(train_x, train_labels)

KeyboardInterrupt: ignored

In [ ]:
# predict with voting-ensembled model and check accuracy on testing set
ypred=vot_hard.predict(test_x)
accuracy_score(test_labels, ypred)

## Voting++

In [ ]:
# Construct the reference table
ref1 = {0: 0.84, 1: 1.0, 2: 0.8, 3: 0.89, 4: 0.8, 5: 0.99, 6: 0.72, 7: 0.95, 8: 0.98, 9: 0.97}
ref2 = {0: 0.82, 1: 1.0, 2: 0.71, 3: 0.81, 4: 0.71, 5: 0.89, 6: 0.56, 7: 0.88, 8: 0.94, 9: 0.92}
ref3 = {0: 0.79, 1: 0.99, 2: 0.8, 3: 0.88, 4: 0.83, 5: 0.84, 6: 0.66, 7: 0.94, 8: 0.97, 9: 0.96}
ref_df = [ref1, ref2, ref3]

In [ ]:
ypred_org = y_pred_test
ypred = ypred_org.copy()

In [ ]:
# represent the y_test_pred with according precision
for i in range(ypred.shape[1]):
   ypred.iloc[:,i] = ypred.iloc[:,i].replace(ref_df[i])

ypred.columns = [1,2,3]
ypred

In [ ]:
# For each sample, take the result from model with highest precision 
n=ypred.idxmax(axis='columns')
result=[]
for j in range(len(n)):
    m=n[j]-1
    result.append(ypred_org.iloc[j,m])

In [ ]:
# Calculate the precision for udf model
udf_report=metrics.classification_report(test_labels, result, output_dict=True)
pd.DataFrame(udf_report).transpose()

# Stacking

In [ ]:
clf = StackingClassifier(estimators=estimator, final_estimator=LogisticRegression(max_iter=200, solver='liblinear'))
clf.fit(train_x, train_labels).score(test_x, test_labels)